In [18]:
import tensorflow.contrib.learn as skflow
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore


import io
import requests
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [12]:
# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)
    
# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)
    
# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    # Regression
    return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [2]:
path = "./data/"

In [3]:
filename_read = os.path.join(path,"Train_Solar.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

In [6]:
df.head(1)

,datetime,date,time,I,T,UV,WS,RH,P
0,1/1/2017 6:00,1/1/2017,6:00:00,0.0,24.6,0.0,1.2,59.8,0.0


In [7]:
# create feature vector
missing_median(df, 'I')
missing_median(df, 'T')
missing_median(df, 'UV')
missing_median(df, 'WS')
missing_median(df, 'RH')
missing_median(df, 'P')
df.drop('datetime',1,inplace=True)
df.drop('date',1,inplace=True)
df.drop('time',1,inplace=True)

In [9]:
df.head(1)

,I,T,UV,WS,RH,P
0,0.0,24.6,0.0,1.2,59.8,0.0


In [11]:
# Drop outliers in horsepower
print("Length before Power outliers dropped: {}".format(len(df)))
remove_outliers(df,'P',2)
print("Length after Power outliers dropped: {}".format(len(df)))

Length before Power outliers dropped: 131586
Length after Power outliers dropped: 129577


In [13]:
x,y = to_xy(df,"P")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [16]:
print(x[:5])

[[ 0.    24.6    0.     1.2   59.8  ]
 [ 0.    24.5    0.     1.4   60.   ]
 [ 0.    24.5    0.     1.9   60.2  ]
 [ 0.    24.5    0.     1.3   60.3  ]
 [ 0.    24.6    0.     1.4   60.125]]


In [17]:
print(y[:5])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [19]:
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

In [20]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(97182, 5)
(32395, 5)
(97182, 1)
(32395, 1)


In [21]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(25,activation='relu'))
model.add(Dense(1)) # Output

In [22]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [23]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

In [24]:
model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Train on 129577 samples, validate on 32395 samples
Epoch 1/1000
 - 5s - loss: 94.7641 - val_loss: 0.3672
Epoch 2/1000
 - 5s - loss: 0.4122 - val_loss: 4.2579
Epoch 3/1000
 - 5s - loss: 0.5087 - val_loss: 0.2429
Epoch 4/1000
 - 4s - loss: 0.4782 - val_loss: 0.2677
Epoch 5/1000
 - 4s - loss: 0.4495 - val_loss: 0.2526
Epoch 6/1000
 - 5s - loss: 0.4131 - val_loss: 0.3263
Epoch 7/1000
 - 5s - loss: 0.3685 - val_loss: 0.2452
Epoch 8/1000
 - 5s - loss: 0.3525 - val_loss: 0.3566
Epoch 00008: early stopping


In [25]:
from sklearn import metrics
import tensorflow as tf

pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

[[ 0.402533  ]
 [-0.15045965]
 [ 0.9749318 ]
 [ 1.6153617 ]
 [-0.04341489]]


In [26]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.5971640944480896


In [27]:
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(path,"R2_model.h5"))

In [28]:
# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(path,"R2_model.json"), "w") as json_file:
    json_file.write(model_json)

In [29]:
# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(path,"R2_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)